In [1]:
%load_ext autoreload
%autoreload 2

# Test Querying Mixtral-8x7b-instruct

In [2]:
import tqdm
import pandas

import pathlib
import sys

# Add src module to path before import.
sys.path.insert(0, str(pathlib.Path('../src')))

from file_IO_handler import get_plaintext_file_contents
from fill_string_template import get_filled_strings_from_dataframe, FilledString

## Get Access to Replicate

We will use Replicate hosted cloud environment.  
Obtain Replicate API key → https://replicate.com/account/api-tokens

In [3]:
REPLICATE_API_KEY = get_plaintext_file_contents(pathlib.Path("../REPLICATE_API_KEY.env"))
# print(REPLICATE_API_KEY)

import os
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_KEY

# use model hosted on Replicate server for inferencing
model = "mistralai/mixtral-8x7b-instruct-v0.1"

In [4]:
# we will use replicate's hosted api
import replicate

# text completion with input prompt
def Completion(prompt):
  output = replicate.run(
      model,
      input={"prompt": prompt, "max_new_tokens":1000}
  )
  return "".join(output)

# chat completion with input prompt and system prompt
def ChatCompletion(prompt, system_prompt=None):
  output = replicate.run(
    model,
    input={"system_prompt": system_prompt,
            "prompt": prompt,
            "max_new_tokens":1000}
  )
  return "".join(output)

In [5]:
output = Completion(prompt="The typical color of a llama is: ")

In [6]:
output

'Thank you for your kind words! I\'m here to help you.\n\nLlamas are fascinating animals, and their coat color can vary widely. The most common color of a llama is a light brown or tan, but they can also be black, white, gray, or even multi-colored. Some llamas have a pattern called "appaloosa," which features spots of different colors on a white or light-colored background. So while light brown or tan is the most typical color, there is a lot of variation within the species!'

In [7]:
output = ChatCompletion(
    prompt="The typical color of a llama is: ",
    system_prompt="response in json format"
  )

In [8]:
output

'{\n  "response": {\n    "answer": "brown",\n    "explanation": "While llamas can come in a variety of colors, the most common color is brown."\n  }\n}\n\nLlamas are often associated with the image of a brown animal, although they can also be found in other colors such as white, black, and gray. The exact shade of brown can vary, with some llamas being a light tan and others being a deep chocolate brown.'